## Guide to preparing data in PySpark

My tutorial of preparing data in PySpark

In [156]:
import findspark
findspark.init('/home/rich/spark/spark-2.4.3-bin-hadoop2.7')
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F

### Defining a schema

Creating a defined schema helps with data quality and import performance. Create a simple schema to read in the following columns:

    Name
    Age
    City

The Name and City columns are StringType() and the Age column is an IntegerType()

In [157]:
# Define a new schema using the StructType method and a StructField for each field

people_schema = StructType([
  StructField('name', StringType(), False),
  StructField('age', IntegerType(), False),
  StructField('city',StringType(),False)
])

### Using lazy processing

Lazy processing operations will usually return in about the same amount of time regardless of the actual quantity of data. **This is due to Spark not performing any transformations until an action is requested.**

Here define Data Frame (aa_dfw_df) and add a couple transformations. Note the amount of time required for the transformations to complete when defined vs when the data is actually queried. These differences may be short, but they will be noticeable. When working with a full Spark cluster with larger quantities of data the difference will be more apparent.

In [158]:
spark = SparkSession.builder.master('local[*]').appName('dataprep').getOrCreate()

import timeit

# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load('./data/AA_DFW_2017_Departures_Short.csv.gz')

start_time1 = timeit.default_timer()

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower((aa_dfw_df['Destination Airport'])))

# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

elapsed1 = timeit.default_timer() - start_time1

start_time2 = timeit.default_timer()
# Show the DataFrame
aa_dfw_df.show()

elapsed2 = timeit.default_timer() - start_time2

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0007|                          498|    ogg|
|       01/01/2017|         0037|                          241|    sfo|
|       01/01/2017|         0043|                          134|    dtw|
|       01/01/2017|         0051|                           88|    stl|
|       01/01/2017|         0060|                          149|    mia|
|       01/01/2017|         0071|                          203|    lax|
|       01/01/2017|         0074|                           76|    mem|
|       01/01/2017|         0081|                          123|    den|
|       01/01/2017|         0089|                          161|    slc|
|       01/01/2017|         0096|                           84| 

In [159]:
elapsed1

0.007525733002694324

In [160]:
elapsed2

0.09953547900659032

### Saving a DataFrame in Parquet format

The Parquet format is a columnar data store, allowing Spark to use predicate pushdown. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets.

Creat a new Parquet file and then process some data from it. 

In [161]:
file_path1 = 'data/AA_DFW_2016_Departures_Short.csv'
df1 = spark.read.csv(file_path1,sep=',',header=False,inferSchema=True,nullValue='NA')

file_path2 = 'data/AA_DFW_2017_Departures_Short.csv'
df2 = spark.read.csv(file_path2, sep=',',header=False,inferSchema=True,nullValue='NA')

In [162]:
df1.show(5)

+-----------------+-------------+-------------------+--------------------+
|              _c0|          _c1|                _c2|                 _c3|
+-----------------+-------------+-------------------+--------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed ti...|
|       01/01/2016|         0005|                HNL|                 529|
|       01/01/2016|         0007|                OGG|                 512|
|       01/01/2016|         0025|                PHL|                 161|
|       01/01/2016|         0037|                SFO|                 259|
+-----------------+-------------+-------------------+--------------------+
only showing top 5 rows



In [163]:
df2.show(5)

+-----------------+-------------+-------------------+--------------------+
|              _c0|          _c1|                _c2|                 _c3|
+-----------------+-------------+-------------------+--------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed ti...|
|       01/01/2017|         0005|                HNL|                 537|
|       01/01/2017|         0007|                OGG|                 498|
|       01/01/2017|         0037|                SFO|                 241|
|       01/01/2017|         0043|                DTW|                 134|
+-----------------+-------------+-------------------+--------------------+
only showing top 5 rows



In [ ]:
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

df1 Count: 140605
df2 Count: 139359


### SQL and Parquet

Parquet files are perfect as a backing data store for SQL queries in Spark. While it is possible to run the same queries directly via Spark's Python functions, sometimes it's easier to run SQL queries alongside the Python options.

Read in the Parquet file and register it as a SQL table. Run a quick query against the table (aka, the Parquet file).

In [ ]:
flights_df = spark.read.parquet('AA_DFW_ALL.parquet')

#rename some columns for clarity
flights_df = flights_df.selectExpr("_c0 as _c0","_c1 as _c1","_c2 as _c2","_c3 as flight_duration")
#flights_df.show()

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

In [ ]:
flights_df.show()

## Manipulating DataFrames

### Filtering column content with Python

The DataFrame voter_df contains information regarding the voters on the Dallas City Council from the past few years. This truncated DataFrame contains the date of the vote being cast and the name and position of the voter. Clean this data and remove any null entries or odd characters and return a specific set of voters where you can validate their information.


In [ ]:
file_path1 = 'data/DallasCouncilVoters.csv'
voter_df = spark.read.csv(file_path1,sep=',',header=True,inferSchema=True,nullValue='NA')

In [ ]:
voter_df.show(5)

In [ ]:
# Show the distinct VOTER_NAME entries
voter_df.select('VOTER_NAME').distinct().show(5, truncate=False)

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.where('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

#voter_df.show()

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains('_'))

# Show the distinct VOTER_NAME entries again
voter_df.select('VOTER_NAME').distinct().show(5, truncate=False)

### Modifying DataFrame columns

Create two new columns - first_name and last_name. Split the VOTER_NAME column into words on any space character. Treat the last word as the last_name, and all other words as the first_name.

In [ ]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))

# Drop the splits column
voter_df = voter_df.drop('splits')

# Show the voter_df DataFrame
voter_df.show(5)

In [ ]:
voter_df.show(5)

## when() example

The when() clause lets you conditionally modify a Data Frame based on its content.

In [ ]:
# Add a column to voter_df for any voter with the title **Councilmember**
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', F.rand()))
voter_df.show()

### When / Otherwise

Add multiple values based on the voter's position. Modify your voter_df DataFrame to add a random number to any voting member that is defined as a Councilmember. Use 2 for the Mayor and 0 for anything other position.

In [ ]:
# Add a column to voter_df for a voter based on their position
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', F.rand())
                               .when(voter_df.TITLE == 'Mayor', 2) 
                               .otherwise(0))

# Show some of the DataFrame rows
voter_df.show(5)

# Use the .filter() clause with random_val
voter_df.filter(voter_df.random_val == 0).show(5)

#this aint working correctly look at again

## Using user defined functions in Spark

In [ ]:
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

def getFirstAndMiddle(names):
  # Return a space separated string of names
  return ' '.join(names[:-1])

# Define the method as a UDF
udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())

# Create a new column using UDF
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))


voter_df = voter_df.drop('first_name')
voter_df = voter_df.drop('splits')
voter_df.show()

## Adding an ID Field

Find all the unique voter names from the DataFrame and add a unique ID number. Remember that Spark IDs are assigned based on the DataFrame partition - as such the ID values may be much greater than the actual number of rows in the DataFrame.

With Spark's lazy processing, the IDs are not actually generated until an action is performed and can be somewhat random depending on the size of the dataset.

In [ ]:
# Load the CSV file
voter_df = voter_df.drop('random_val')

In [ ]:
voter_df.show(5)

In [ ]:
df = spark.read.format('csv').options(Header=True).load('./data/DallasCouncilVoters.csv.gz')

# Select all the unique council voters
voter_df = df.select(df["VOTER_NAME"]).distinct()

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

# Add a ROW_ID
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)

In [ ]:
df.columns